In [8]:
def infoGain(P,N):
    import math
    return -P/(P+N)*math.log2(P/(P+N))-N/(P+N)*math.log2(N/(P+N))

In [9]:
def insertNode(tree,d,Node):
    for k, v in tree.items():
        if isinstance(v, dict):
            tree[k] = insertNode(v, d, Node)
    if d in tree:
        if isinstance(tree[d], dict):
            tree[d][Node] = 'None'
        else:
            tree[d] = {Node:'None'} 
    return tree

In [10]:
def insertConcept(tree,d,Node):
    for k, v in tree.items():
        if isinstance(v, dict):
            tree[k] = insertConcept(v, d, Node)
    if d in tree:
        tree[d] = Node
    return tree

In [11]:
def getNextData(data,Node,d):
    iData = (data[Node] == d)
    nD = data[iData]
    nD = nD.drop(Node,axis=1)
    return nD

In [12]:
def getNextNode(data,AttributeList,concept,conceptVals,tree,d):
    Total=data.shape[0]
    if Total == 0: #If Attibutes are empty, then return current value of tree
        return tree
    
    countC={}
    for cVal in conceptVals:   #If Example is positive then return positive and If negative then return negative
        dataCC = data[data[concept] == cVal]   #Get data for specific concept
        countC[cVal] = dataCC.shape[0]         #Get the count of data for specific concept

        if countC[cVal] == 0 and cVal == 'No': #If all examples are Positive(Not Negative), return single node Positive
            tree = insertConcept(tree,d,'Yes')
            return tree
        if countC[cVal] == 0 and cVal == 'Yes': #If all examples are Negative (Not Positive), return single node Negative
            tree = insertConcept(tree,d,'No')
            return tree
    
    ClassEntropy = infoGain(countC['Yes'],countC['No']) #Class Entropy for d
    
    Attr = {}         #Attribute dict holding list of possible values
    for a in AttributeList:
        Attr[a] = list(set(data[a]))
        
    AttrCount = {}    #Get the attribute values being positive and negative
    TotalInfo = {}    #Inomation Gain temp dict Total for all attr
    InfoGain = {}     #Inomation Gain temp dict Total for each attr
    EntropyAttr = {}  #Attribute Entropy
    for att in AttributeList:
        for vals in Attr[att]:
            for c in conceptVals:
                iData = (data[att] == vals)      #Get data for specific attribute
                cData = (data[concept] == c)     #Get data for specific concept
                dataAtt = data[iData][cData]     #Get data for specific attribute and concept
                AttrCount[c] = dataAtt.shape[0]  #Get the count of data for specific attribute and concept
            TotalInfo[vals] = AttrCount['Yes'] + AttrCount['No'] #Total Attribute 
            if AttrCount['Yes'] == 0 or AttrCount['No'] == 0:
                InfoGain[vals]=0
            else:
                InfoGain[vals] = infoGain(AttrCount['Yes'],AttrCount['No']) #Calculate InfoGain for each  attr
            
            if att not in EntropyAttr:          #Calculate Entropy for each attr
                EntropyAttr[att] = ( TotalInfo[vals] / Total ) * InfoGain[vals] 
            else:
                EntropyAttr[att] = EntropyAttr[att] + ( TotalInfo[vals] / Total ) * InfoGain[vals]
    
    Gain = {}
    for g in EntropyAttr:
        Gain[g] = ClassEntropy - EntropyAttr[g] #Calculate gain
#     print("Gain : ",Gain)
    Node = max(Gain,key=Gain.get)               #Get root node
#     print("Next root : ",Node)
    tree = insertNode(tree,d,Node)
    for nD in Attr[Node]:
        tree = insertNode(tree,Node,nD) #Insert 
        newData = getNextData(data,Node,nD)
        AttributeList=list(newData)[:-1]
        tree = getNextNode(newData,AttributeList,concept,conceptVals,tree,nD)
    return tree

In [13]:
def main():
    import pandas as pd
    from pandas import DataFrame 
    data = DataFrame.from_csv('software hardware.csv')    #Read CSV
    AttributeList=list(data)[:-1]            #Get Attribute List
    concept=str(list(data)[-1])              #Get concept list
    conceptVals = list(set(data[concept]))   #Get Concept values
    tree = getNextNode(data,AttributeList,concept,conceptVals,{'root':'None'},'root')
    print(tree)

In [14]:
main()

{'root': {'Age': {'Mid': {'Competition': {'No': 'Yes', 'Yes': 'No'}}, 'Old': 'No', 'New': 'Yes'}}}


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
